**Collaborative filtering** solves the content-based filtering problem by starting with only incomplete user ratings data—no predefined item features or user preference surveys are needed. Instead, it learns both user factors and item factors automatically through an iterative process that improves predictions over repeated steps.

### Contrast with content-based filtering

Content-based filtering requires **known item factors** (like genre tags or expert ratings) to infer user factors via linear regression per user. The "reverse" approach requires **known user factors** (like survey responses on music styles) to infer item factors via linear regression per item. Both need some upfront content metadata, which is hard to obtain at scale

### How collaborative filtering works

**Core idea**: Treat both users and items symmetrically—learn latent factors for both simultaneously using only the ratings matrix.

1. **Start with random item factors**  
   - Assign each item $k$ random numeric factors (e.g., $k=2$: factor1, factor2).  
   - These have no meaning—they're just starting points (e.g., Ommazh: [0.3, -0.7]).

2. **Infer user factors (user regression step)**  
   - For each user, fit linear regression: predict their known ratings using the current (random) item factors.  
   - The learned regression coefficients become that user's factors (e.g., An: θ₁=-0.165, θ₂=3.98).  
   - Prediction formula: rating = θ_user · factors_item (no intercept for simplicity).

3. **Infer item factors (item regression step)**  
   - For each item, fit linear regression: predict its known ratings using the current user factors.  
   - The learned coefficients become that item's updated factors.

4. **Repeat (alternating optimization)**  
   - Use new item factors → infer new user factors → use new user factors → infer new item factors.  
   - Each full cycle reduces squared error between predicted and actual ratings.  
   - Stop when error converges or after fixed iterations.

### Why this converges (no black magic)

- It's **alternating least squares (ALS)** optimization of matrix factorization.  
- Ratings matrix $R$ ≈ $U × V^T$, where $U$ = user factors matrix, $V$ = item factors matrix.  
- Each step minimizes loss holding the other matrix fixed, like coordinate descent.  
- The ratings data constrains the factors to make good predictions, so random starts evolve toward meaningful latent representations.

### Key characteristics

**Advantages over content-based**:
- No manual feature engineering—factors emerge from data patterns.
- Scales to millions of items/users without content metadata.
- "Collaborative" because users' collective ratings reveal item similarities.

**Properties of learned factors**:
- Number of factors ($k$) is a hyperparameter (try 2, 10, 50+).
- Factors are usually uninterpretable (not "lo-fi indie," just abstract dimensions).
- Handles sparse data naturally (missing ratings = no constraint).

**Prediction for unseen items**:
- Once converged: predicted_rating(user, item) = dot_product(user_factors, item_factors).
- Rank items by predicted score to recommend top ones.

This approach powers Netflix, Amazon, Spotify—any system with user-item interaction data but limited item metadata.